In [ ]:
import copy # so we can make deepcopies of the board to test out diff algorithms

import random # to help generate random boards

In [ ]:
# it wont always create a board with a possible solution

def create_board():

  board = [ [ 0 for _ in range(9) ] for _ in range(9) ]

  for i in range(9):

      for j in range(9):

          k = random.randint(1, 9)

          if is_valid(board, i, j, k): # to make sure that there are no duplicates within row, col, or region
          # even if there is no solution but atleast the board should be valid
            board[i][j] = k

  return board

In [ ]:
def is_valid(board, row, col, num):

    if num in board[row]: #check rows
        return False

    if num in [board[i][col] for i in range(9)]: #check columns
        return False

    box_row, box_col = 3 * (row // 3), 3 * (col // 3)
    # row // 3 gives quotient so we know if we are in first second or third region
    # then mult by 3 to get the row index
    # similarly for column


    if num in [board[box_row + i][box_col + j] for i in range(3) for j in range(3)]:
        return False # just go thru three rows and columns to check the region

    return True

In [ ]:
def print_board(board):
    for i in range(9):

        if i%3==0 and i!=0:
                  print("---------------------")

        for j in range(9):

            if j%3==0 and j!=0:
                print("|", end=" ")

            if board[i][j] == 0:
                print("_", end=" ") #print empty space instead of 0

            else:
                print(board[i][j], end=" ")


        print()

In [ ]:
# even tho we do have a create_board function
# this is a sample board with one valid solution
# so we're just going to use this for testing
# to make sure the programs give a solution when one exists

sudoku_board = [
    [3, 0, 6, 5, 0, 8, 4, 0, 0],
		[5, 2, 0, 0, 0, 0, 0, 0, 0],
		[0, 8, 7, 0, 0, 0, 0, 3, 1],
		[0, 0, 3, 0, 1, 0, 0, 8, 0],
		[9, 0, 0, 8, 6, 3, 0, 0, 5],
		[0, 5, 0, 0, 9, 0, 6, 0, 0],
		[1, 3, 0, 0, 0, 0, 2, 5, 0],
		[0, 0, 0, 0, 0, 0, 0, 7, 4],
		[0, 0, 5, 2, 0, 6, 3, 0, 0]]

Minimum Remaining Value

In [ ]:
def count_valid_values(board, row, col):
    count = 0

    for num in range(1, 10):
        if is_valid(board, row, col, num):
            count += 1

    return count

  # just checks how many of the values between 1 and 10 are valid at some position (row, col)

  # we use this for mrv to find the place that that has the fewest valid values ie SMALLEST DOMAIN

In [ ]:
def find_empty_location_MRV(board):

    min_values = float('inf') # just initializing it to infinity so that the first min val we get is always smaller
    # if we make this zero it willl never find a smaller value

    min_location = None

    for i in range(9):

        for j in range(9):

            if board[i][j] == 0:
                valid_values = count_valid_values(board, i, j) # the number of valid values that can be placed at an empty spot

                if valid_values < min_values: # if smaller than a previously found then replace it and dave the location
                    min_values = valid_values
                    min_location = (i, j)

    return min_location

def solve_sudoku_MRV(board):

    location = find_empty_location_MRV(board) # for mrv we start with the location having least number of valid values

    if location is None:
        return True # solution is found

    # we have to check if location is None before extracting the row and col
    # otherwise it will give error
    # "cant unpack NoneType"

    row, col = location

    if row is None:
        return True # solution is found

    for num in range(1, 10):
      # check all values and see if they are valid

      if is_valid(board, row, col, num):

          board[row][col] = num

          if solve_sudoku_MRV(board): # recursive call
              return True  # solution found

          board[row][col] = 0  # backtrack

    return False


if __name__ == "__main__":

    # Generate a random Sudoku puzzle
    #sudoku_board = create_board()

    # for now to test the programs we are just gonna use this one kyunke we know it has a solution
    board_mrv = copy.deepcopy(sudoku_board)

    print("\npuzzle")
    print_board(board_mrv)

    if solve_sudoku_MRV(board_mrv):
        print("\nsolution:")
        print_board(board_mrv)
    else:
        print("\nno solution")



puzzle
3 _ 6 | 5 _ 8 | 4 _ _ 
5 2 _ | _ _ _ | _ _ _ 
_ 8 7 | _ _ _ | _ 3 1 
---------------------
_ _ 3 | _ 1 _ | _ 8 _ 
9 _ _ | 8 6 3 | _ _ 5 
_ 5 _ | _ 9 _ | 6 _ _ 
---------------------
1 3 _ | _ _ _ | 2 5 _ 
_ _ _ | _ _ _ | _ 7 4 
_ _ 5 | 2 _ 6 | 3 _ _ 

solution:
3 1 6 | 5 7 8 | 4 9 2 
5 2 9 | 1 3 4 | 7 6 8 
4 8 7 | 6 2 9 | 5 3 1 
---------------------
2 6 3 | 4 1 5 | 9 8 7 
9 7 4 | 8 6 3 | 1 2 5 
8 5 1 | 7 9 2 | 6 4 3 
---------------------
1 3 8 | 9 4 7 | 2 5 6 
6 9 2 | 3 5 1 | 8 7 4 
7 4 5 | 2 8 6 | 3 1 9 


Degree Heuristic

In [ ]:
# for this the base code is mostly same as mrv but we use degree heuristic for tiebreaker

def find_empty_location_DH(board):

    min_values = float('inf')# just initializing it to infinity so that the first min val we get is always smaller
    # if we make this zero it willl never find a smaller value

    min_location = None

    for i in range(9):

        for j in range(9):

            if board[i][j] == 0:
                valid_values = count_valid_values(board, i, j)

                if valid_values < min_values:
                    min_values = valid_values
                    min_location = (i, j)
                # yahan tak it was the same as mrv

                # now we see what to do if we get a tie
                elif valid_values == min_values:

                    # degree hueristic tie breaker
                    # we check degrees for both positions and use the greater degree
                    degree1 = count_degree(board, i, j)
                    degree2 = count_degree(board, min_location[0], min_location[1])

                    # we will prefer the position that affects a greater number of other positions
                    if degree1 > degree2:
                        min_location = (i, j)

    return min_location


def count_degree(board, row, col):
    degree = 0

    for i in range(9):
      # check how many empty spots that row has
        if board[row][i] == 0: # and is_valid(board, row, i, col):
            degree += 1

      # check how many empty spots that column has
        if board[i][col] == 0: # and is_valid(board, i, col, row):
            degree += 1

    # check how many empty spots that region has
    # row // 3 gives quotient so we know if we are in first second or third region
    # then mult by 3 to get the row index
    # similarly for column
    box_row, box_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(box_row, box_row + 3):
        for j in range(box_col, box_col + 3):
            if board[i][j] == 0: #and is_valid(board, i, j, row):
                degree += 1

    # now we know how many other empty spots are in the same row col and region
    # ie the number of constraints on other unassigned variables
    # so the degree is how many other variables it will affect
    return degree

def solve_sudoku_DH(board):

    # logic for this main solve function for dh is pretty much same as mrv
    location = find_empty_location_DH(board)
    if location is None:
        return True

    row, col = location

    for num in range(1, 10):
        if is_valid(board, row, col, num):
            board[row][col] = num

            if solve_sudoku_DH(board):
                return True

            board[row][col] = 0

    return False



if __name__ == "__main__":
    # Generate a random Sudoku puzzle
    #sudoku_board = create_board()

    # for now to test the programs we are just gonna use this one because we know it has a solution
    board_dh = copy.deepcopy(sudoku_board)

    print("Generated Sudoku Puzzle:")
    print_board(board_dh)

    if solve_sudoku_DH(board_dh):
        print("\nSolution:")
        print_board(board_dh)
    else:
        print("\nNo solution exists for the given puzzle.")


Generated Sudoku Puzzle:
3 _ 6 | 5 _ 8 | 4 _ _ 
5 2 _ | _ _ _ | _ _ _ 
_ 8 7 | _ _ _ | _ 3 1 
---------------------
_ _ 3 | _ 1 _ | _ 8 _ 
9 _ _ | 8 6 3 | _ _ 5 
_ 5 _ | _ 9 _ | 6 _ _ 
---------------------
1 3 _ | _ _ _ | 2 5 _ 
_ _ _ | _ _ _ | _ 7 4 
_ _ 5 | 2 _ 6 | 3 _ _ 

Solution:
3 1 6 | 5 7 8 | 4 9 2 
5 2 9 | 1 3 4 | 7 6 8 
4 8 7 | 6 2 9 | 5 3 1 
---------------------
2 6 3 | 4 1 5 | 9 8 7 
9 7 4 | 8 6 3 | 1 2 5 
8 5 1 | 7 9 2 | 6 4 3 
---------------------
1 3 8 | 9 4 7 | 2 5 6 
6 9 2 | 3 5 1 | 8 7 4 
7 4 5 | 2 8 6 | 3 1 9 


Least Constraining Value

In [ ]:
def find_empty_location_LCV(board):
  # this time we jsut use the first empty spot that we find
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                return (row, col)
    return None

def lcv_values(board, row, col):
    num_constraints = {}

    for num in range(1, 10):

      # check if a number can be placed in empty spot
        if is_valid(board, row, col, num):
            constraints = 0

            # constraints = that number is valid for how many other empty spots on the board?
            # ie; by placing this number we are constraining how many other spots
            # we want the value that would limit other variable's domains the least

            # check constraints in row
            for j in range(9):
                if board[row][j] == 0 and j != col and is_valid(board, row, j, num):
                    constraints += 1

            # check constraints in column
            for i in range(9):
                if board[i][col] == 0 and i != row and is_valid(board, i, col, num):
                    constraints += 1

            # similar logic as before for getting the small region
            start_row, start_col = 3 * (row // 3), 3 * (col // 3)

            # check constraints in the 3x3 region
            for i in range(3):
                for j in range(3):
                    if board[start_row + i][start_col + j] == 0 and (start_row + i != row or start_col + j != col):
                        if is_valid(board, start_row + i, start_col + j, num):
                            constraints += 1

            # store the constraints for this num at index num
            num_constraints[num] = constraints

    # sorts the numbers based on their constraints in ascending order
    # so that, in solve_sudoku_lcv, we can start checking from the number with least constraints
    sorted_nums = sorted(num_constraints.keys(), key=lambda x: num_constraints[x])
    return sorted_nums

def solve_sudoku_LCV(board):
    location =  find_empty_location_LCV(board)

    if location is None:
        return True

    row, col = location

    # here instead of checking all values in 0 to 9 sequence
    # we check values starting with the least constraining ones
    for num in lcv_values(board, row, col):

      # same backtracking logic for the rest
        if is_valid(board, row, col, num):
            board[row][col] = num

            if solve_sudoku_LCV(board):
                return True

            board[row][col] = 0

    return False

if __name__ == "__main__":
    # Generate a random Sudoku puzzle
    #sudoku_board = create_board()

    # for now to test the programs we are just gonna use this one because we know it has a solution
    board_lcv = copy.deepcopy(sudoku_board)

    print("Generated Sudoku Puzzle:")
    print_board(board_lcv)

    if solve_sudoku_LCV(board_lcv):
        print("\nSolution:")
        print_board(board_lcv)
    else:
        print("\nNo solution exists for the given puzzle.")


Generated Sudoku Puzzle:
3 _ 6 | 5 _ 8 | 4 _ _ 
5 2 _ | _ _ _ | _ _ _ 
_ 8 7 | _ _ _ | _ 3 1 
---------------------
_ _ 3 | _ 1 _ | _ 8 _ 
9 _ _ | 8 6 3 | _ _ 5 
_ 5 _ | _ 9 _ | 6 _ _ 
---------------------
1 3 _ | _ _ _ | 2 5 _ 
_ _ _ | _ _ _ | _ 7 4 
_ _ 5 | 2 _ 6 | 3 _ _ 

Solution:
3 1 6 | 5 7 8 | 4 9 2 
5 2 9 | 1 3 4 | 7 6 8 
4 8 7 | 6 2 9 | 5 3 1 
---------------------
2 6 3 | 4 1 5 | 9 8 7 
9 7 4 | 8 6 3 | 1 2 5 
8 5 1 | 7 9 2 | 6 4 3 
---------------------
1 3 8 | 9 4 7 | 2 5 6 
6 9 2 | 3 5 1 | 8 7 4 
7 4 5 | 2 8 6 | 3 1 9 
